[View in Colaboratory](https://colab.research.google.com/github/georgegacitua/EL4106-Inteligencia_Computacional/blob/master/convnet-tutorial.ipynb)

### Subir código cifar10.py. Basta con ejecutar una vez (inmune a reinicios de kernel de python, no de la máquina)

In [1]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving cifar10.py to cifar10.py
User uploaded file "cifar10.py" with length 6585 bytes


# Aquí comienza el cuerpo del código
## Reinicie luego de entrenar cada modelo. Recuerde actualizar el directorio de tensorboard.

In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import sys
import time

import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline 
import tensorflow as tf

from cifar10 import CIFAR10

In [0]:
tf.set_random_seed(1)
sess = tf.Session()

### Constructor del dataset. Modifique aquí el *flag* de *data augmentation*

In [4]:
# Load dataset
batch_size = 64
cifar10 = CIFAR10(batch_size=batch_size, validation_proportion=0.1, augment_data=False)

In [0]:
# Model blocks
def conv_layer(input_tensor, kernel_shape, layer_name):
    # input_tensor b01c
    # kernel_shape 01-in-out
    weights = tf.get_variable("weights", kernel_shape,
                               initializer = tf.contrib.layers.xavier_initializer_conv2d())
    biases = tf.get_variable("biases", [kernel_shape[3]],
                             initializer=tf.constant_initializer(0.05))
    
    tf.summary.histogram(layer_name + "/weights", weights)
    tf.summary.histogram(layer_name + "/biases", biases)
    
    # Other options are to use He et. al init. for weights and 0.01 
    # to init. biases.
    conv = tf.nn.conv2d(input_tensor, weights, 
                       strides = [1, 1, 1, 1], padding='SAME')
    return tf.nn.relu(conv + biases)

def fc_layer(input_tensor, weights_shape, layer_name):
    # weights_shape in-out
    weights = tf.get_variable("weights", weights_shape,
                              initializer = tf.contrib.layers.xavier_initializer())
    biases = tf.get_variable("biases", [weights_shape[1]],
                             initializer=tf.constant_initializer(0.0))
    tf.summary.histogram(layer_name + "/weights", weights)
    tf.summary.histogram(layer_name + "/biases", biases)
    mult_out = tf.matmul(input_tensor, weights)
    return tf.nn.relu(mult_out+biases)
    

## Elija aquí el directorio donde guardará los registros de Tensorboard

In [0]:
PARENT_DIR = './summaries/'
SUMMARIES_DIR = PARENT_DIR + 'conv_2_layer_with_dropout'

## Construcción del grafo

### Construcción del modelo + función de costos

In [7]:
# Model
use_convnet = True
n_conv_layers = 2

n_filters_convs = [16, 32, 64]

model_input = tf.placeholder(tf.float32, name='model_input', 
                             shape=(batch_size, 32, 32, 3))

keep_prob = tf.placeholder(tf.float32, name='dropout_prob', shape=())

target = tf.placeholder(tf.float32, name='target', shape=(batch_size, 10))

if use_convnet:
    layer_input = model_input
    previous_n_feature_maps = 3
    for layer_index in range(n_conv_layers):
      layer_name = 'conv%d' % layer_index
      with tf.variable_scope(layer_name):
        conv_out = conv_layer(
            layer_input, 
            [5, 5, previous_n_feature_maps, n_filters_convs[layer_index]], 
            layer_name)
      if layer_index == 0:
        with tf.variable_scope(layer_name, reuse=True):
          conv1_filters = tf.get_variable("weights")
          tf.summary.image(
              'conv1_filters',
              tf.transpose(conv1_filters, perm=[3, 0, 1, 2]),
              max_outputs=n_filters_convs[layer_index]
          )
      previous_n_feature_maps = n_filters_convs[layer_index]
      pool_out = tf.nn.max_pool(
          conv_out, 
          ksize=[1, 2, 2, 1],
          strides=[1, 2, 2, 1],
          padding='SAME',
          name='pool%d' % layer_index)
      layer_input = pool_out
     

    fc_input = tf.layers.flatten(pool_out, name='fc_input')

    feature_map_height = int(32 / (2**n_conv_layers))
    
    # First fully connected layer
    layer_name = 'fc1'
    with tf.variable_scope(layer_name):
        fc1_out = fc_layer(
            fc_input, 
            [(feature_map_height**2)*previous_n_feature_maps, 50], 
            layer_name)

    fc1_out_drop = tf.nn.dropout(fc1_out, keep_prob)

    # Second fully connected layer
    layer_name = 'fc2'
    with tf.variable_scope(layer_name):
        fc2_out = fc_layer(fc1_out_drop, [50, 10], layer_name)
    model_output = fc2_out
        
else:
    # Reshape tensor to MLP
    first_layer_input = tf.reshape(model_input, [-1,3072], name='first_layer_input')

    # First layer
    layer_name = 'fc1'
    with tf.variable_scope(layer_name):
        fc1_out = fc_layer(first_layer_input, [3072, 100], layer_name)

    fc1_out_drop = tf.nn.dropout(fc1_out, keep_prob)

    # Second layer
    layer_name = 'fc2'
    with tf.variable_scope(layer_name):
        fc2_out = fc_layer(fc1_out_drop, [100, 10], layer_name)
    model_output = fc2_out

with tf.name_scope('loss_function'):
    cross_entropy = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits=model_output, labels=target,
                                           name='cross_entropy'))
    xentropy_summary = tf.summary.scalar('cross_entropy', cross_entropy)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



### Construcción del optimizador + funciones auxiliares

In [0]:
# Optimization
with tf.name_scope('optimizer'):
    optimizer = tf.train.RMSPropOptimizer(0.0005)
    grads_vars = optimizer.compute_gradients(cross_entropy)
    optimizer.apply_gradients(grads_vars)
    train_step = optimizer.minimize(cross_entropy)

# Metrics
correct_prediction = tf.equal(tf.argmax(model_output, 1),
                             tf.argmax(target, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name='accuracy')
accuracy_summary = tf.summary.scalar('accuracy', accuracy)
learning_summaries = tf.summary.merge((xentropy_summary, accuracy_summary))
merged = tf.summary.merge_all()

# Useful training functions
def validate():
    cifar10.shuffleValidation()
    batches = cifar10.getValidationSet(asBatches=True)
    accs = []
    xent_vals = []
    for batch in batches:
        data, labels = batch
        acc, xentropy_val = sess.run((accuracy, cross_entropy),
                       feed_dict={
                model_input: data,
                target: labels,
                keep_prob: 1.0
            })
        accs.append(acc)
        xent_vals.append(xentropy_val)
    mean_xent = np.array(xent_vals).mean()    
    mean_acc = np.array(accs).mean()
    summary = sess.run(
        merged,
        feed_dict={
            model_input: data,
            target: labels,
            keep_prob: 1.0
        })
    return summary, mean_acc, mean_xent
def test():
    batches = cifar10.getTestSet(asBatches=True)
    accs = []
    for batch in batches:
        data, labels = batch
        acc = sess.run(accuracy,
                       feed_dict={
                model_input: data,
                target: labels,
                keep_prob: 1.0
            })
        accs.append(acc)
    mean_acc = np.array(accs).mean()
    return mean_acc

# Tensorboard writers
train_writer = tf.summary.FileWriter(SUMMARIES_DIR+'/train',
                                     sess.graph)
validation_writer = tf.summary.FileWriter(SUMMARIES_DIR+'/validation')

## Entrenar
### Modifique el valor de *keep_prob* usado al computar *train_step* para activar/desactivar el *dropout*.
### NO MODIFIQUE EL KEEP_PROB USADO EN OTROS SITIOS, ESOS CORRESPONDEN A INFERENCIA.

In [9]:
sess.run(tf.global_variables_initializer())
cifar10.reset()
print("Trainable variables")
for n in tf.trainable_variables():
    print(n.name)
if use_convnet:
    epochs = 30
else:
    epochs = 50
    
t_i = time.time()
n_batches = cifar10.n_batches
val_acc_vals = []
test_acc_vals = []
while cifar10.getEpoch() < epochs:
    epoch = cifar10.getEpoch()
    batch, batch_idx = cifar10.nextBatch()
    batch_data = batch[0]
    batch_labels = batch[1]
    
    # just a training iteration
    _ = sess.run(train_step,
                feed_dict={
            model_input: batch_data,
            target: batch_labels,
            keep_prob: 0.5 ###### Modifique el dropout aqui y solo aqui. #####
        })
    
    step = batch_idx+epoch*n_batches
    
    # Write training summary
    if step%50==0:
        summary = sess.run(learning_summaries,
                          feed_dict={
                model_input: batch_data,
                target: batch_labels,
                keep_prob: 1.0 # set to 1.0 at inference time
            })
        train_writer.add_summary(summary, step)
        
    # gradient (by layer) statistics over last training batch & validation summary
    if batch_idx==0:
        loss, acc, grads = sess.run((cross_entropy, accuracy, grads_vars), 
                      feed_dict={
            model_input: batch_data,
            target: batch_labels,
            keep_prob: 1.0
        })
        
        summary, validation_accuracy, validation_loss = validate()
        validation_writer.add_summary(summary, step)
        print('[Epoch %d, it %d] Training acc. %.3f, loss %.3f. \
Valid. acc. %.3f, loss %.3f' % (
            epoch,
            step,
            acc,
            loss,
            validation_accuracy,
            validation_loss
        ))
        val_acc_vals.append(validation_accuracy)
        test_accuracy = test()
        test_acc_vals.append(test_accuracy)
        print("Time elapsed %.2f minutes" % ((time.time()-t_i)/60.0))
train_writer.flush()
validation_writer.flush()

val_acc_vals = np.array(val_acc_vals)
test_acc_vals = np.array(test_acc_vals)
best_epoch = np.argmax(val_acc_vals)
test_acc_at_best = test_acc_vals[best_epoch]
print('*'*30)
print("Testing set accuracy @ epoch %d (best validation acc): %.4f" % (best_epoch, test_acc_at_best))
print('*'*30)

Trainable variables
conv0/weights:0
conv0/biases:0
conv1/weights:0
conv1/biases:0
fc1/weights:0
fc1/biases:0
fc2/weights:0
fc2/biases:0
[Epoch 0, it 0] Training acc. 0.141, loss 2.329. Valid. acc. 0.090, loss 2.345
Time elapsed 0.03 minutes
[Epoch 1, it 703] Training acc. 0.453, loss 1.560. Valid. acc. 0.467, loss 1.525
Time elapsed 0.12 minutes
[Epoch 2, it 1406] Training acc. 0.719, loss 0.998. Valid. acc. 0.565, loss 1.263
Time elapsed 0.21 minutes
[Epoch 3, it 2109] Training acc. 0.719, loss 0.943. Valid. acc. 0.582, loss 1.200
Time elapsed 0.29 minutes
[Epoch 4, it 2812] Training acc. 0.656, loss 1.017. Valid. acc. 0.622, loss 1.119
Time elapsed 0.38 minutes
[Epoch 5, it 3515] Training acc. 0.750, loss 0.748. Valid. acc. 0.640, loss 1.056
Time elapsed 0.46 minutes
[Epoch 6, it 4218] Training acc. 0.797, loss 0.874. Valid. acc. 0.651, loss 1.004
Time elapsed 0.55 minutes
[Epoch 7, it 4921] Training acc. 0.734, loss 0.706. Valid. acc. 0.667, loss 0.975
Time elapsed 0.63 minutes
[Epo

## Visualizar en Tensorboard

In [10]:
# ----- Descarga de ngrok para crear tunel
%%bash
file="ngrok-stable-linux-amd64.zip"
if [ -f "$file" ]
then
	echo "$file already downloaded."
else
    wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
    unzip ngrok-stable-linux-amd64.zip
fi

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


--2018-10-19 01:49:49--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.2.175.150, 52.201.75.180, 52.0.94.50, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.2.175.150|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5363700 (5.1M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

     0K .......... .......... .......... .......... ..........  0%  794K 7s
    50K .......... .......... .......... .......... ..........  1% 1.57M 5s
   100K .......... .......... .......... .......... ..........  2% 31.8M 3s
   150K .......... .......... .......... .......... ..........  3% 69.5M 2s
   200K .......... .......... .......... .......... ..........  4% 1.64M 3s
   250K .......... .......... .......... .......... ..........  5% 67.1M 2s
   300K .......... .......... .......... .......... ..........  6% 59.9M 2s
   350K .......... .......... .......... .......... ..........  7% 62

In [11]:
# ----- Ejecutar despues de nueva corrida para actualizar Tensorboard
print("Showing summaries at %s" % (PARENT_DIR))

get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(PARENT_DIR)
)
get_ipython().system_raw('./ngrok http 6006 &')
print('Click URL to open TensorBoard:')
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

Showing summaries at ./summaries/
Click URL to open TensorBoard:
http://68a79835.ngrok.io


In [12]:
%%bash
ls
ps aux | grep -e 'tensorboard'

cifar-10-batches-py
cifar10.py
ngrok
ngrok-stable-linux-amd64.zip
__pycache__
sample_data
summaries
root         150  5.1  2.1 2028280 284400 ?      Sl   01:49   0:03 /usr/bin/python2 /usr/local/bin/tensorboard --logdir ./summaries/ --host 0.0.0.0 --port 6006
